# Quick warmup

In [53]:
class Clock():
    def __init__(self, n=24):
        self.time = 0
        self.cycles = 0
        self.n = n
    
    def tick(self):
        self.time +=1
        if self.time >= self.n:
            self.time = 0
        self.cycles +=1
    
    def __eq__(self, other):
        return self.time == other.time
    
    def __repr__(self):
        return f"The time is {self.time}:00"
        

In [55]:
a = Clock()
b = Clock()
print(a==b)

for i in range(50):
    print(a)
    a.tick()

print(a==b)

True
The time is 0:00
The time is 1:00
The time is 2:00
The time is 3:00
The time is 4:00
The time is 5:00
The time is 6:00
The time is 7:00
The time is 8:00
The time is 9:00
The time is 10:00
The time is 11:00
The time is 12:00
The time is 13:00
The time is 14:00
The time is 15:00
The time is 16:00
The time is 17:00
The time is 18:00
The time is 19:00
The time is 20:00
The time is 21:00
The time is 22:00
The time is 23:00
The time is 0:00
The time is 1:00
The time is 2:00
The time is 3:00
The time is 4:00
The time is 5:00
The time is 6:00
The time is 7:00
The time is 8:00
The time is 9:00
The time is 10:00
The time is 11:00
The time is 12:00
The time is 13:00
The time is 14:00
The time is 15:00
The time is 16:00
The time is 17:00
The time is 18:00
The time is 19:00
The time is 20:00
The time is 21:00
The time is 22:00
The time is 23:00
The time is 0:00
The time is 1:00
False


# Exercise Set 4: controlling for observable factors and causal trees

In this Exercise Set 4 we will try out different techniques for using matching and try an implementation of causal trees. 

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns
from scipy.stats import ttest_rel, ttest_ind

%matplotlib inline

<br>

## 4.1 Survival and passenger class

We revisit a classic dataset: Titanic. We are interested in analyzing whether the passengers on First class had a higher survival probability. 

The code below loads the dataset. 

In [222]:
df = sns.load_dataset('titanic').dropna(subset=['age'])


X = pd.get_dummies(df.drop(['pclass','class', 'alive'],axis=1), drop_first=True).astype('float')
D = (df['pclass'] < 3).rename('high_class')
y = (df['alive']=='yes').astype('float')

> **Ex. 4.1.1:** Compute the ATE of not travelling on a 3rd class ticket, assuming the CIA holds.

In [67]:
# Your answer here
def ate(y, D):
    y_1 = y[D].mean()
    y_0 = y[~D].mean()
    return y_1-y_0

ate(y, D)

0.33159402095021384

> **Ex. 4.1.2:** Compute the share of males, the proportion travelling alone, and the mean age, by treatment status. Then modify the code below to try out coarsened exact matching on `exact_cols = ['age_group', 'alone','sex_male']` with age bins of size 2, 5, 10 and 15 years. 
>
> Comment on the result. Does coarse matching seem like a feasible approach in this 
```python
age_diff =  2
X['age_group'] =  (X.age // age_diff)
match_count = \
    pd.DataFrame({'treat':X[D].groupby(exact_cols).size(), 
                  'control':X[~D].groupby(exact_cols).size()})\            
n_obs_matched = int(match_count.dropna().sum().sum())
```

In [26]:
# Your answer here
cols = ['age', 'sex_male', 'alone']
print("treated", X[D][cols].mean())
print("Not treated", X[~D][cols].mean())

treated age         34.206825
sex_male     0.557103
alone        0.498607
dtype: float64
Not treated age         25.140620
sex_male     0.712676
alone        0.633803
dtype: float64


In [72]:
# CEM
exact_cols = ['age_group', 'alone','sex_male']
for age_diff in [2,5,10,15]:
    X['age_group'] =  X.age // age_diff
    coarse = pd.DataFrame({'treat':X[D].groupby(exact_cols).size(), 
                  'control':X[~D].groupby(exact_cols).size()})
    print(coarse)

                          treat  control
age_group alone sex_male                
0.0       0.0   0.0         NaN      4.0
                1.0         6.0      4.0
1.0       0.0   0.0         3.0      5.0
                1.0         3.0      5.0
2.0       0.0   0.0         3.0      5.0
...                         ...      ...
33.0      1.0   1.0         1.0      NaN
35.0      0.0   1.0         1.0      NaN
          1.0   1.0         3.0      1.0
37.0      1.0   1.0         NaN      1.0
40.0      1.0   1.0         1.0      NaN

[118 rows x 2 columns]
                          treat  control
age_group alone sex_male                
0.0       0.0   0.0         5.0     12.0
                1.0        10.0     13.0
1.0       0.0   0.0         4.0      8.0
                1.0         1.0      8.0
          1.0   0.0         NaN      1.0
2.0       0.0   0.0         3.0      4.0
                1.0         1.0      5.0
          1.0   0.0         NaN      2.0
                1.0         NaN  

Continue with age difference = 5.

> **Ex. 4.1.3:** Compute the average treatment effect by using (coarsened) exact matching on `age` (i.e. on `age_group`). 
>
>Comment on the result. How does the group treatment effects compare to the ATE you found in 4.1.2?

In [115]:
# Your answer here
cols = ['age_group', 'treated']
X['age_group'] =  X.age // 5
coarse = X.copy()
coarse['treated'] = D.astype(int)
coarse.set_index(cols, inplace=True)
coarse = coarse['survived']
y_coarse = coarse.groupby(cols).mean().unstack('treated')
ate = y_coarse[1] - y_coarse[0]

In [116]:
ate.mean()

0.3597437016811178

> **Ex. 4.1.4:** Estimate a logistic regression model for predicting the passenger class variable (i.e. `D`, the treatment indicator).

In [189]:
# Your answer here
from sklearn.linear_model import LogisticRegression
# deselect highly correlated columns (age, age_group) (man, woman)
cols = ['age', 'sibsp', 'parch', 'fare', 'adult_male', 'alone',
       'sex_male', 'embarked_Q', 'embarked_S', 'deck_B', 'deck_C', 'deck_D', 'deck_E', 'deck_F', 'deck_G',
       'embark_town_Queenstown', 'embark_town_Southampton']

# lr with lasso to better deal with the 20 dimensions
clf = LogisticRegression(random_state=0, max_iter=1000, n_jobs=-1, penalty='l2').fit(X[cols], D)
X['prop'] = clf.predict_proba(X[cols])[:,0]
clf.score(X[cols], D)

0.876750700280112

> **Ex. 4.1.5:** What other models might we have chosen? 

Since we are moddeling a binary variable we could instead have used treebased models such as a random forrest. Alternatively a distance based method like k-nearest neighbors or support vector machines.

In [167]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
svc = LinearSVC(C=1.0, max_iter=1000).fit(X[cols], D)
rfc = RandomForestClassifier(n_jobs=-1).fit(X[cols], D)
nbrs = KNeighborsClassifier().fit(X[cols], D)

print(f"SVC: {svc.score(X[cols], D)}, RFC: {rfc.score(X[cols], D)}, Neighbors: {nbrs.score(X[cols], D)}")

C:\Users\Tobias\Miniconda3\envs\sds_eml_2020\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


SVC: 0.8291316526610645, RFC: 1.0, Neighbors: 0.9355742296918768


> **Ex. 4.1.6:** What is the overlap of predicted probabilities? What happens if you estimate the model without `fare` and `deck`? Comment
>
> Why do `fare` and `deck` matter a lot in this setting, try to draw a causal diagram that might illuminate your discussion.

In [174]:
# Your answer here
cols2 = ['age', 'sibsp', 'parch', 'adult_male', 'alone',
       'sex_male', 'embarked_Q', 'embarked_S',
       'embark_town_Queenstown', 'embark_town_Southampton']

# lr with lasso to better deal with the 20 dimensions
clf = LogisticRegression(random_state=0, max_iter=1000, n_jobs=-1, penalty='l2').fit(X[cols2], D)
clf.score(X[cols2], D)

0.6932773109243697

How much they paid and which deck they were on is highly correlated with which class they where on, because: 
 * i) higher class tickets are more expensive in general, and 
 * ii) lower class cabins are often placed differently from the highter ones (e.g lower in ship)

> **Ex. 4.1.7:** Use a 5-nearest-neighbors matching in propensity space to compute the average treatment effect. Bootstrap the 95 pct. confidence interval of the ATE. What happens if you select only propensity score values with high common support, i.e. between 0.2 and 0.8?

In [226]:
# Your answer here
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(n_neighbors=5, n_jobs=-1)
nn.fit(propensity)  # propenbsity calculated from the first LR
indicies = nn.kneighbors(propensity[D],n_neighbors=5, return_distance=False)
y.iloc[indicies[0]]

1      1.0
385    0.0
445    1.0
72     0.0
540    1.0
Name: alive, dtype: float64

> **Ex. 4.1.7:** (BONUS) How might we improve on the approach above?

In [210]:
# Your answer here
X.shape

(714, 22)

## 4.2 Honest trees

In this problem we will try to implement and understand some of the ideas used in [Athey, Imbens (2015)](https://www.pnas.org/content/pnas/113/27/7353.full.pdf) to develop _Honest Inference_ in desicion tree models. The paper begins by covering honesty in a setting of population averages, and for estimating conditional means; so you will need to look towards the second half of the paper to get an impression of it's use for treatment-effect estimation.

> **Ex. 4.2.1:** What does it mean that a tree is _honest?_ In particular what are the implications in terms of 
> * The intuition for why honesty is required in order to get good local treatment effect estimates?
> * The practical implementation of the DT algorithm?

In [ ]:
# Your answer here

> **Ex.4.2.2:** Use the `load_42_data` function to load the boston house-price dataset. Split your dataset in two. A 50% test set and a 50% train set using `sklearn.model_selection.train_test_split`. 

In [ ]:
def load_42_data():
    from sklearn.datasets import load_boston
    df = load_boston()
    df = pd.DataFrame(np.c_[df['data'], df['target']], columns = list(df['feature_names']) + ['y'])
    return df

In [ ]:
# Your answer here

> **Ex 4.2.3:** Identify the column and value in `X_train` that minimizes the (cross split weighted) sum of squared errors in the training data. Split the test data according to this value and report the mean and standard deviation of `y` in both subsamples for both the train and test data.
>
> Comment on your results. How different are the two subsamples from the overall mean and standard deviation?

In [ ]:
# Your answer here

> **Ex 4.2.4:** Redo your analysis from 4.2.3, but this time split in a 66% train dataset and a 33% test dataset. Split the train data once more 50/50 to get a train and an estimation dataset. 
>
> Focus only on one of the subsets (i.e. either the left or right leaf). 
>
> Report the same statistics as before, but for train, estimation and test samples. This time, show your results as density plots graphing 5.000 bootstrap replications of the whole procedure. If your pc is slow, you might need to reduce the number of replications to 1000.


In [ ]:
# Your answer here